# Intro

The nice thing about the NCAA basketball championship is that we have plenty of dats available. The format of the tournament was changed in 1985, but that still leaves us with 35 years of data. The Kaggle competition has been going on for 6 years now. So there is also plenty of information on Kaggel. This Notebook was inspired by a few of them, so thanks to the Kaggle community for all the valuable information shared!

We set up the basics first.

In [ ]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
%matplotlib inline

# this is the base directory so you can switch between local, Kaggle Notebook, etc.
data_directory = "../input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1"

# Evaluate and Compare

The basic idea of this notebook is that with all the historic data available we can evaluate our algorithms easily. We just us the appropriate data and compare the performance of a variety of possible solutions on all available historic tournaments.

We are doing this by creating a set of predict functions which given a season and two teams give the probability that the first team wins. This can then be applied to the actual brackets for the finals and we can calculate the logloss of our potential submission in historic competitions.

The actual format for submissions are different since not all games played are known beforehand. Therefore in the submission we have to give all possible combinations. Our predict function is generic and can also be used for the actual submission. Only the games actually played are considered for the overall logloss so we calculate the actual logloss by looking at the games played in former tournaments and their results.

## Known Unkowns

If we do know nothing about the teams we can always assume a 50% win probability for each team. So this is a benchmark for our algorithm. Scoring worth than that means something went awfully wrong. 

In [ ]:
# this is a template for the core function
# predict the probability that in a given season (year) lteam beats gteam 
def predict_dummy(season, lteam_id, gteam_id):
    return(0.5)

predict_dummy(2019, 1115, 1118)

### What Really Happened

We test our predictions on the actual tourney results available from 1985 onwards. First we load them and remove the play ins since they are not scored. They are marked as days 134 and 135 in the tourney results.

We will use this data frame and add all our predictions so we can later compare the logloss.

In [ ]:
tourney_results = pd.read_csv(os.path.join(data_directory,'MNCAATourneyCompactResults.csv'))

tourney_results = tourney_results.loc[(tourney_results["DayNum"] != 134) & (tourney_results["DayNum"] != 135)]

tourney_results.index = range(len(tourney_results.index))

tourney_results

The ground truth is is easily created since the tourney results are always given as winning team first and loosing team second. So the result is always 1 for the first team winning. We apply our dummy predictions of 0.5 as a separate column and also test our logic for applying the prediction function.

This is it, our first prediction is ready. But we can do better.

In [ ]:
tourney_results["Truth"] = (tourney_results["WScore"] > tourney_results["LScore"]).astype("float")

tourney_results["Pred_Dummy"] = \
    tourney_results.apply(lambda row: 
                          predict_dummy(row['Season'], row['WTeamID'], row["LTeamID"]), 
                          axis=1)

tourney_results

## Get the Experts

Among the multitude of data files we get there are also the tournament seeds. The seeds are a ranking of teams generated by experts before the tournament to prevent strong teams knocking each other out in the early stages of the tournament. So probably they give some reasonable hint on which team might win a game. It is a bit more complicated since the teams are ranked separately for different areas. To get the numerical seeds we need to parse them from the string. For predictions we are actually interested in the seed differences, so we calculate them. On Kaggle you find formulas for calculating the win probability from the seed difference.

In [ ]:
seeds = pd.read_csv(os.path.join(data_directory,'MNCAATourneySeeds.csv'))

seeds['SeedInt'] = [int(x[1:3]) for x in seeds['Seed']]

# Merge the seeds file with itself on Season.  This creates every combination of two teams by season.
seed_diff = seeds.merge(seeds, how='inner', on='Season')

seed_diff['ID'] = seed_diff['Season'].astype(str) + '_' \
              + seed_diff['TeamID_x'].astype(str) + '_' \
              + seed_diff['TeamID_y'].astype(str)

# formula found on Kaggle for getting probabilities out of seed differences
seed_diff['Pred'] = 0.5 + 0.030975*(seed_diff['SeedInt_y'] - seed_diff['SeedInt_x'])

seed_diff

Using the information calculated above based on the seed information our predict function just has to select the right probability for two teams in a given season.

In [ ]:
def predict_seed(season, lteam_id, gteam_id):
    return(seed_diff.loc[(seed_diff.TeamID_x == lteam_id) &
                 (seed_diff.TeamID_y == gteam_id) &
                 (seed_diff.Season == season)].Pred.mean())

predict_seed(1985, 1207, 1210)

As with our trivial prediction we apply the function to all relevant historic matches.

In [ ]:
tourney_results["Pred_Seed"] = \
    tourney_results.apply(lambda row: 
                          predict_seed(row['Season'], row['WTeamID'], row["LTeamID"]), 
                          axis=1)

tourney_results

## Chess is like Basketball

At least as far as the relevance of Elo ratings is concerned. Elo as a measurement of strength for chess players is heavily used in match making in professional chess. Since the only input parameter are wins of player A against player B it can be applied to many areas of sports. Let's try with NCAA basketball. We need some basic functions for calculating the Elo values which I copied from yet another interesting notebook on Kaggle.

In [ ]:
# based on https://www.kaggle.com/lpkirwin/fivethirtyeight-elo-ratings

# a parameter for elo changes
elo_k = 20
# the home advantage
elo_home = 100

# functions taken for Elo calculation
# this function calculates a wind probability from the elo ratings
def elo_pred(elo1, elo2):
    return(1. / (10. ** (-(elo1 - elo2) / 400.) + 1.))

# this calculates an expexted score difference for a given elo difference
def expected_margin(elo_diff):
    return((7.5 + 0.006 * elo_diff))

# this calculates the update (and the prediction) taking into account home advantage
def elo_update(w_elo, l_elo, margin, wloc):
    if wloc == "H":
        w_elo += elo_home
    elif wloc == "A":
        l_elo += elo_home
    elo_diff = w_elo - l_elo
    pred = elo_pred(w_elo, l_elo)
    mult = ((margin + 3.) ** 0.8) / expected_margin(elo_diff)
    update = elo_k * mult * (1 - pred)
    return(pred, update)

For actually calculating the Elo values we still have some alternatives. Like the notebook I based this on we are calculating them on all the season games. Each season adjusts the Elo values from the end of the previous season so we calculate a rating based on the history of the team back to 1985. Since players change frequently in college basketball some people might consider this not necessarily as the best idea.

In [ ]:
season_results = pd.read_csv(os.path.join(data_directory,'MRegularSeasonCompactResults.csv'))

season_results["ScoreDiff"] = season_results.WScore - season_results.LScore
season_results["WElo"] = 0
season_results["LElo"] = 0

season_results

To make the code a bit simpler and probably also more efficient we use a cache for the current Elo of all the teams. Based on this cache we look at each match, update the cache and store the current after match Elo of both teams with the match. Note that this might take some time as we are looking at quite a few games. You can restrict the seasons if it takes to long. 

In [ ]:
# We initialise the ELOs to 1500 before the season
team_ids = set(season_results.WTeamID).union(set(season_results.LTeamID))

# cache for the current elo of all teams
current_elo = dict(zip(list(team_ids), [1500] * len(team_ids)))

for index, row in season_results.iterrows():
    pred, update = elo_update(current_elo[row.WTeamID],
                              current_elo[row.LTeamID],
                              row.ScoreDiff,
                              row.WLoc)
    current_elo[row.WTeamID] += update
    current_elo[row.LTeamID] -= update
    season_results.loc[index, "WElo"] = current_elo[row.WTeamID]
    season_results.loc[index, "LElo"] = current_elo[row.LTeamID]

season_results

While this is a data structure well suited to analyse how Elo affected match outcomes for our purpose it is not practical. The challenge is that we have to search the teams in two columns. So we use another data frame where we just have teams and the Elo. This is also well suited for analysing historic development of team strengths. 

However for the prediction we are only interested in the Elo at the end of each season for all teams. This we can get by looking at the Elo at the maximum day for each season for each team. Sounds more complicated than it actually is. 

In [ ]:
elo_summary = season_results[["Season", "DayNum", "WTeamID", "WElo"]].copy()
elo_summary.rename(columns={"WTeamID" : "LTeamID", "WElo" : "LElo"}, inplace=True)
elo_summary = pd.concat((elo_summary, season_results[["Season", "DayNum", "LTeamID", "LElo"]]), axis=0, ignore_index=True)
elo_summary.rename(columns = {"LTeamID" : "TeamID", "LElo" : "Elo"}, inplace=True)

idx = elo_summary[(elo_summary.Season == 2015) & (elo_summary.TeamID == 1433)]["DayNum"].idxmax()
elo_summary.iloc[idx].Elo

Using all this our predict function is quite compact. We againapply it to all tournament matches. Since we only used information before the tournament this also works in the actual Kaggle competition.

In [ ]:
def predict_elo(season, lteam_id, gteam_id):
    idx_l = elo_summary[(elo_summary.Season == season) & (elo_summary.TeamID == lteam_id)]["DayNum"].idxmax()
    idx_g = elo_summary[(elo_summary.Season == season) & (elo_summary.TeamID == gteam_id)]["DayNum"].idxmax()

    return(elo_pred(elo_summary.iloc[idx_l].Elo, elo_summary.iloc[idx_g].Elo))

predict_elo(1985, 1207, 1210)

In [ ]:
tourney_results["Pred_Elo"] = \
    tourney_results.apply(lambda row: 
                          predict_elo(row['Season'], row['WTeamID'], row["LTeamID"]), 
                          axis=1)

tourney_results

## And the Winner is ...

Now our tourney_results data frame also has our predictions using the different prediction algorithms we have tried. Now the question is which one would have performed best had we actually submitted in that particular season. Kaggle uses logloss to calculate the scores for the leaderboard. It is calculated per game and can then be averaged to get the actual result.

So first we define two functions to calculate the logloss as specified by Kaggle in the competition rules. Again I found it in a Notebook on Kaggle.

In [ ]:
def kaggle_clip_log(x):
    '''
    Calculates the natural logarithm, but with the argument clipped within [1e-15, 1 - 1e-15]
    '''
    return np.log(np.clip(x,1.0e-15, 1.0 - 1.0e-15))

def kaggle_log_loss(pred, result):
    '''
    Calculates the kaggle log loss for prediction pred given result result
    '''
    return -(result*kaggle_clip_log(pred) + (1-result)*kaggle_clip_log(1.0 - pred))

Now we just calculate the logloss per game for each of our predictions and list the mean per season. This would have been our score had we submitted. As mentioned before: It is only fair if we only use data already available just before the actual tournament had taken place. This is the case for both the seed information and our way of calculating the Elo.

In [ ]:
tourney_results["LogLoss_Dummy"] = kaggle_log_loss(tourney_results["Pred_Dummy"], tourney_results["Truth"])
tourney_results["LogLoss_Seed"] = kaggle_log_loss(tourney_results["Pred_Seed"], tourney_results["Truth"])
tourney_results["LogLoss_Elo"] = kaggle_log_loss(tourney_results["Pred_Elo"], tourney_results["Truth"])

tourney_results.groupby('Season').agg({'LogLoss_Dummy' : 'mean', 'LogLoss_Seed' : 'mean', 
                                       "LogLoss_Elo" : "mean"})

## Here's Looking at Your Results

Finally we plot our prediction results. You can easily see that algorithms like these perform better some seasons and worse others. A difference in logloss of 0.1 is quite a lot! Fortunately for sports fans everywhere the games do not always follow the logic of team strength. If in a given tournament there were more surprise winners our predictions are significantly worse as opposed to the favorites taking the game.

Hopefully you can use this code to evaluate you predictions and find more clever and better performaning ones than the examples given here. Please upvote if you found it useful and comment if you have suggestions.

Good luck for the competition!

In [ ]:
df_plt = tourney_results.groupby('Season').agg({'LogLoss_Dummy' : 'mean', 'LogLoss_Seed' : 'mean', 
                                                "LogLoss_Elo" : "mean"})

plt.plot(df_plt.LogLoss_Seed, label="Seed")
plt.plot(df_plt.LogLoss_Elo, label="Elo")
plt.axhline(df_plt[["LogLoss_Seed", "LogLoss_Elo"]].values.mean())
plt.legend()
plt.show()

## Creating a Submission

After all this work you can easily create a submission based on the sample submission provided by Kaggle. We parse out our parameters for the predict function and make our prediction. For the submission we only need the ID and the prediction. Good luck in the competition!

In [ ]:
submission = pd.read_csv(os.path.join(data_directory,'../MSampleSubmissionStage1_2020.csv'))

# parse year and the two teams out of it
submission['season'] = [int(x[0:4]) for x in submission['ID']]
submission['lteam'] = [int(x[5:9]) for x in submission['ID']]
submission['gteam'] = [int(x[10:14]) for x in submission['ID']]

submission["Pred"] = \
    submission.apply(lambda row: 
                          predict_elo(row['season'], row['lteam'], row["gteam"]), 
                          axis=1)

submission

In [ ]:
submission[["ID", "Pred"]].to_csv(os.path.join(data_directory, "/kaggle/working/my_submission.csv"), index=False)